# <center>AdaBoost</center>

## 一、集成学习

### 1.1 定义

　　所谓集成学习(ensemble learning),是指通过构建多个弱学习器,然后结合为一个强学习器来完成分类任务.并相较于弱分类器而言,进一步提升结果的准确率.严格来说,集成学习并不算是一种分类器,而是一种学习器结合的方法.

　　集成学习的整个流程:首次按产生一组"个体学习器",这些个体学习器可以是同质的(homogeneous)(例如全部是决策树),这一类学习器被称为基学习器(base learner),相应的学习算法称为"基学习算法";集成也可包含不同类型的个体学习器(例如同时包含决策树和神经网络),这一类学习器被称为"组件学习器"(component learner)

　　集成学习通过将多个学习器进行结合,可获得比单一学习器显著优越的泛华性能,它基于这样一个思想:对于一个复杂任务来说,将多个专家的判断进行适当的综合所得出的判断,要比其中任何一个专家单独的判断好.

　　对于提升方法来说,有两个问题需要回答:<font color='red'>一是在每一轮如何改变训练数据的权值或概率分布;二是如何将弱分类器组合成一个强分类器</font>.关于第一个问题,AdaBoost的做法是,提高那些被前一轮弱分类器错误分类样本的权值,而降低那些被正确分类样本的权值.这样依赖,那些没有得到正确分类的数据,由于其权值的加大而受到后一轮的弱分类器的更大关注.对于第二个问题,即弱分类器的组合,AdaBoost采取加权多数表决的方法.具体地址,加大分类误差率小的弱分类器的权值,使其在表决中起较大的作用,减少分类误差率的弱分类器的权值,使其在表决中其较小的作用.

### 1.2 集成学习的分类

　　当前,我们可以立足于通过处理数据集生成差异性分类器,即在原有数据集上采用抽样技术获得多个训练数据集来生成多个差异性分类器.根据个体学习器的生成方式,目前集成学习方法大致可以分为两大类:第一类是个体学习器之间存在强依赖关系、必须串行生成的序列化方法,这种方法的代表是"Boosting";第二类是个体学习器间不存在强依赖关系、可同时生成的并行化方法,它的代表是"Bagging"和"Random Forest"

- Bagging: 通过对原数据进行有放回的抽取,构建出多个样本数据集,然后用这些新的数据训练集训练多个分类器.因为是有放回的采用,所以一些样本可能会出现多次,而其他样本会被忽略.该方法是通过降低基分类器方法来改善泛化能力,因此Bagging的性能依赖与基分类器的稳定性,如果基分类器是不稳定的,Bagging有助于降低训练数据的随机扰动导致的误差,但是如果基分类器是稳定的,即对数据变化不敏感,那么Bagging方法就得不到性能的提升,甚至会降低.

- Boosting:提升方法是一个迭代的过程,通过改变样本分布,使得分类器聚集在那些很难分的样本上,对那些容易错分的数据加强学习,增加错分数据的权重,这样错分的数据在下一轮的迭代就有更大的作用(对错分数据进行惩罚)

- Boosting与Bagging的区别

-- 两者的主要区别是采样方式不同.Bagging采用均匀取样,而Boosting根据错误率来采样,因此Boost的分类精度要由于Bagging.Bagging的训练集的选择是随机的,各轮训练集之间相互独立,而Boosting的各轮训练集的选择与前面各轮的学习结果有关;Bagging的各个预测函数没有权重,而Boosting是有权重的;Bagging的各个预测函数可以并行生成,而Boosting的个预测函数只能顺序生成.对于像神经网络这样极为耗时的学习方法,Bagging可通过并行训练节省大量时间开销  
  
-- bagging是减少variance,而boosting是减少bias.Bagging是Bootstrap Aggregating的简称,意思就是再取样(Bootstrap),然后在每个样本上训练出来的模型取平均,所以是降低模型的variance. Bagging比如Random Forest这样先天并行的算法都有这个效果. Boosting则是迭代算法,每一次迭代都根据上一次迭代的预测结果对样本进行加权,所以随着迭代不断进行,误差会越来越小,所以模型的bias会不断降低.这种算法无法并行.

## 一、AdaBoost算法

　　现在叙述AdaBoost算法.假定给定一个二分类的训练数据集

$$T=\{(x_1,y_1),(x_2,y_2),\cdots,(x_n,y_n)\}$$

其中$y_i$属于二分类的标记组合,即$y_i \in \{+1,-1\}$,AdaBoost算法是从训练数据中学习一系列弱分类器或基本分类器,并将这些弱分类器线性组合成一个强分类器

- 步骤一: 首先,初始化训练数据的权值分布.假设每一个训练样本最开始时都被赋予相同的权值:$\frac{1}{N}$,即每个训练样本在基分类器的学习中作用相同,这一假设保证步骤一能够在原始数据上学习基本分类器$G_1(x)$数学化的语言表示为:

$$D_1=(w_{11},w_{12},\cdots,w_{1i},\cdots,w_{1N})\ ,\ w_{1i}=\frac{1}{N}\ ,\ i=1,2,\cdots,N$$

- 步骤二: AdaBoost反复学习基本分类器,在每一轮$m=1,2,\cdots,M$顺序执行下列操作

 1) 使用当前权值分布为$D_m$的训练数据集,学习得到基分类

$$G_m(x): \lambda \to \{-1,+1\}$$

2) 计算上一步得到的基分类器$G_m(x)$在训练数据集上的分类误差率$e_m$为

$$e_m=P(G_m(x)\neq y_i)=\frac{\sum_{i=1}^Nw_{mi}I(G_m(x_i)\neq y_i)}{\sum_{i=1}^Nw_{mi}}=\sum_{i=1}^Nw_{mi}I(G_m(x_i)\neq y_i)$$

　　这里$w_{mi}$表示第$m$轮中第$i$个实例的权值,$\sum_{i=1}^Nw_{mi}=1$.这表明,$G_m(x)$在加权的训练数据集上的分类误差率是被$G_m(x)$误分类样本的权值之和,由此可以看出数据权值分布$D_m$与基本分类器$G_m(x)$的分类误差率的关系.

3) 计算$G_m$前面的权重系数$\alpha_m$,该系数表示$G_m$在最终分类器中的重要程度,目的在于使我们得到基分类器在最终分类器中所占的权重,系数计算公式如下:

$$\alpha_m=\frac{1}{2}\log\frac{1-e_m}{e_m}$$

　　这里的对数是自然对数,由表达式可知,当$e\leq \frac{1}{2}$时,$\alpha_m \geq 0$,并且$\alpha_m$随着$e_m$的减小而增大,意味着分类误差率越小的基分类器在最终分类器中的作用越大,而$e \geq \frac{1}{2}$刚刚相反,这正好验证了集成学习中每个个体分类器的分类精度必须大于0.5的前提条件

4) 更新训练数据集的权值分布为下一轮作准备

$$D_{m+1}=(w_{m+1,1},w_{m+1,2},\cdots,w_{m+1,i},\cdots,w_{m+1,N})$$

其中

$$w_{m+1,i}=\frac{w_{mi}}{Z_m}\exp(-\alpha_my_iG_m(x_i))\ ,\ i=1,2,\cdots,N$$

　　由此可知,被基本分类器$G_m(x)$误分类样本的权值得以扩大,而被正确分类样本的权值得以缩小.两两比较,误分类样本的权值$t^{2\alpha_m}=\frac{e_m}{1-e_m}$倍.因此,误分类样本在下一轮学习中起更大的作用.不改变所给的训练数据,而不断改变训练数据权值的分布,使得训练数据在基本分类器的学习中起不同的作用,这时AdaBoost的一个特点.  
　　这里我们还引入了一个规范化因子,它的作用在于使$D_{m+1}$成为一个概率分布.具体公式为

$$Z_m=\sum_{i=1}^Nw_{mi}exp(-\alpha_my_iG_m(x_i))\ ,\ i=1,2,\cdots,N$$

　　重复步骤二中的1至4步骤,得到一系列的权重参数$\alpha_m$和基分类器$G_m$

- 步骤三: 将上一步得到的基分类器根据权重参数线性组合

$$f(x)=\sum_{m=1}^M\alpha_mG_m(x)$$

线性组合$f(x)$实现了M个基本分类器的加权表决.系数$\alpha_m$表示了基本分类器$G_m(x)$的重要性,这里,所有的$\alpha_m$之和并不为1.$f(x)$的符号决定实例$x$的类,f(x)的绝对值表示分类的确信度,利用基本分类器的线性组合构建最终分类器是Adaboost的另一特点.

## 二、AdaBoost算法的一个实例

　　下图为给定的训练样本,假设$Y \in \{+1,-1\}$,且弱分类器由$xv产生(v$为阈值,目的在于使分类器在训练样本上的分类误差率最低),接下来我们就要使用AdaBoost算法得到一个强分类器.

\| 序号 \| 1 \| 2 \| 3 \| 4 \| 5 \| 6 \| 7 \| 8 \| 9 \| 10 \|
\| x \| 0 \| 1 \| 2 \| 3 \| 4 \| 5 \| 6 \| 7 \| 8 \| 9 \|
\| y \| 1 \| 1 \| 1 \| -1 \| -1 \| -1 \| 1 \| 1 \| 1 \| -1 \|

　　首先,初始化训练数据的权值分布,得到

$$D_1=(w_{1,1},w_{1,2},\cdots,w_{i,10})\ ,\ w_{1i}=\frac{1}{10}\ ,\ i=1,2,\cdots,10$$

　　在此基础上,开始M轮迭代

　　根据X和Y的对应关系,要把这10个数据分为两类,一类是1,一类是-1,根据数据的特地发现:(0,1,2,6,7,8)对应的类是1,(3,4,5,9)对应的类是-1,抛开孤立的9不说,(0,1,2),(3,4,5),(6,7,8)这是3类不同的数据,分别对应的类是(1,-1,1),直观上推测可知,可以找到对应的数据分界点,比如2.5、5.5、8.5,将这几类数据分成两类.

1.第一次迭代(m=1):

- 1) 在第一次迭代时,已知$w_{1,i}=\frac{1}{10}$,经计算可得:在权值分布为$D_1$的训练数据上,阈值$v$取2.5或8.5时分类误差率为0.3,取5.5时分类误差率为0.4,遵循分类误差率最低原则,从2.5或8.5中任意选取一个阈值,这里选取2.5,故基本分类器为

$$G_1(x)=\{_{-1\ x>2.5}^{1\ x<2.5}$$

- 2) $G_1(x)$在训练集上的误差率:

$$e_1=P(G_1(x_i)\neq y_i)=0.3$$

- 3) 根据$e_1$计算得到$G_1(x)$的系数:

$$\alpha_1 = \frac{1}{2}\log \frac{1-e_1}{e_1}=0.4236$$

　　这个系数就代表$G_1(x)$在最终的分类函数中所占的权值

- 4) 更新训练数据的权值分布,用于下一轮迭代

$$D_{m+1}=(w_{m+1,1},w_{m+1,2},\cdots,w_{m+1,i},\cdots,w_{m+1,N})$$

其中

$$w_{m+1,i}=\frac{w_{mi}}{Z_m}\exp(-\alpha_my_iG_m(x_i))\ ,\ i=1,2,\cdots,N$$

　　由此得到$D_2=(0.0715,0.0715,0.715,0.0715,0.0715,0.715,0.166,0.166,0.166,0.0715)$,根据$D_2$可知,分队的样本权重由0.1下降到0.0715,分错的样本(6,7,8)的权值由0.1上升至0.166.

　　此时分类函数为$sign[f_1(x)]=sign[0.4236G_1(x)]$

2.第二次迭代(m=2):

- 1) 在上一轮迭代中,我们获知了新一轮的权重分布$D_2$,在此基础上,经过计算可得,阈值$v$是8.5时分类误差率最低,因此第二个基本分类器为

$$G_2(x)=\{_{-1\ \ x>8.5}^{1\ \ x<8.5}$$

- 2) 误差率:

$$e_2=P(G_2(x_i)\neq y_i = 0.0715×3=0.2143$$

- 3) $G_2(x)的系数为$:

$$\alpha_2=\frac{1}{2}\log \frac{1-e_2}{e_2}=0.6496$$

- 4) 更新训练数据的权值分布,得到

$D_3=(0.0455,0.0455,0.0455,0.1667，0.1667,0.1667,0.1060,0.1060,0.1060,0.0455)$,相较于$D_2$,被分错的样本(3,4,5)的权值变大,其他被分队的样本的权值变小.经过第二轮迭代后,分类函数为$sign[f_2(x)]=sign[0.4236G_1(x)+0.6496G_2(x)]$.将10个样本点依次代入到第二个分类器中,可得到此时依然有着3个误分类点(3,4,5),为此需要进行下一轮迭代

3.第三次迭代(m=3):

- 1) 在上一轮迭代中,我们获知了新一轮的权重分布$D_3$,在此基础上,经过计算可得,阈值$v$是5.5时分类误差率最低,因此第三个基本分类器为

$$G_2(x)=\{_{1\ \ x>5.5}^{-1\ \ x<5.5}$$

- 2) 误差率:

$$e_2=P(G_2(x_i)\neq y_i = 0.0455×3=0.1820$$

- 3) $G_2(x)的系数为$:

$$\alpha_2=\frac{1}{2}\log \frac{1-e_2}{e_2}=0.7514$$

- 4) 更新训练数据的权值分布,得到

$D_4=(0.125，0.125，0.125，0.102，0.102，0.102，0.065，0.065，0.065，0.125)$,此时分类函数为$sign[f_3(x)]=sign[0.4236G_1(x)+0.6496G_2(x)+0.7514G_3(x)]$.同样将10个样本点一次代入第三个分类器中,可发现没有误分类点,全部样本点已正确分类,因此停止迭代.算法运行完毕.

## 三、AdaBoost算法的训练误差分析

　　AdaBoost最基本的性质是它能在学习过程中不断减小训练误差,即在训练数据集上的分类误差率.关于这个问题有下面的定理

### 3.1 AdaBoost的训练误差界

　　首先,给出AdaBoost的训练误差界的定理:

$$\frac{1}{N}\sum_{i=1}^{N}I(G(x_i \neq y_i)) \leq \frac{1}{N}\sum_i \exp(-y_if(x_i))=\prod _m Z_m$$

　　其中

$$G(x)=sign(f(x))=sign(\sum_{m=1}^M \alpha_mG_m(x))$$

$$Z_m=\sum_{i=1}^Nw_{mi}\exp(-\alpha_my_iG_m(x_i))$$

$$f(x)=\sum_{m=1}^M \alpha_mG_m(x)$$

证明如下:

- 1) 当$G(x_i)\neq y_i$时,$y_if(x_i)<0$,因而$\exp(-y_if(x_i)) \geq 1$.由此直接推导出前半部分.

- 2) 后半部分的推导需要用到

$$w_{m+1,i}=\frac{w_{mi}}{Z_m} \exp (-\alpha_m y_i G_m(x_i))\ ,\ i=1,2,\cdots,N$$

$$w_{mi}\exp(-\alpha_m y_i G_m(x_i))=Z_mw_{m+1,i}$$

推导如下

$$\frac{1}{N}\sum_i \exp(-y_if(x_i))$$

$$=\frac{1}{N}\sum_i \exp(-\sum_{m=1}^M \alpha_m y_i G_m(x_i))$$

$$=\sum_i w_{1i}\prod_{m=1}^M \exp(-\alpha_m y_i G_m(x_i))$$

$$=Z_1\sum_i w_{2i}\prod_{m=1}^M \exp(-\alpha_m y_i G_m(x_i))$$

$$=Z_1Z_2\sum_i w_{3i}\prod_{m=1}^M \exp(-\alpha_m y_i G_m(x_i))$$

$$=Z_1Z_2 \cdots Z_{M-1}\sum_i w_{Mi}\prod_{m=1}^M \exp(-\alpha_M y_i G_m(x_i))$$

$$=\prod_{m=1}^M Z_m$$

　　<font color='red'>这一定理说明,可以在每一轮选取适当的$G_m$使得$Z_m$最小,从而使训练误差下降最快</font>

### 3.2 二分类问题AdaBoost的训练误差界

对于二分类问题,有如下结果

$$\prod_{m=1}^MZ_m=\prod_{m=1}^M[2\sqrt{e_m(1-em)}]=\prod_{m=1}^M \sqrt{(1-4\gamma_m^2)} \leq \exp(-2 \sum_{m=1}^M \gamma_m^2)$$

这里

$$\gamma_m = \frac{1}{2} - e_m$$

证明:

- 1) 首先,等式部分

$$Z_m=\sum_{i=1}^N w_{mi} \exp(-\alpha_m y_i G_m(x_i))$$

$$=\sum_{y_i=G_m(x_i)} w_{mi} \exp(-\alpha_m) + \sum_{y_i \neq G_m(x_i)}w_{mi}\exp(\alpha_m)$$

$$=(1-e_m)e^{-\alpha_m} + e_me^{\alpha_m}$$

$$=2 \sqrt{e_m(1-e_m)}$$

$$=\sqrt{1-4\gamma_m^2}$$

- 2) 不等式部分,先由$e^x$和$\sqrt{(1-x)}$在$x=0$处的泰勒展开式

$$e^x=1+x+\frac{1}{2}x^2+\cdots+\frac{1}{n!}x^n+O(x^n)$$

$$(1-x^{\frac{1}{2}})=1-\frac{1}{2}x - \frac{1}{8}x^2+\cdots+O(x^n)$$

推出不等式

$$\sqrt{(1-4\gamma_m^2)} \leq \exp(-2\gamma_m^2)$$

进而得到.

### 3.3 推论

　　由上述两个定理推出,如果存在$\gamma >0$,对所有m有$\gamma_m \geq \gamma$,则

$$\frac{1}{N}\sum_{i=1}^NI(G(x_i)\neq y_i) \leq \exp(-2M\gamma^2)$$

这个结论表明在此条件下AdaBoost的训练误差是以指数速率下降的.

## 四、AdaBoost算法的数学推导

　　AdaBoost算法还有另一个解释,AdaBoost算法可以被认为模型是加法模型,损失函数为指数函数、学习算法为前向分布算法的二分类学习方法

### 4.1 前向分布算法

在推导之前,先敲定几个概念

- 加法模型(additive model)

$$f(x)=\sum_{m=1}^M \beta_mb(x;\gamma_m)$$

　　其中$b(x;\gamma_m)$为基函数,$\beta_m$为基函数的系数.

- 损失函数极小化:给定训练数据及损失函数$L(y,f(x))$的条件下,学习加法模型$f(x)$称为经验风险极小化即损失函数极小

$$\min_{\beta_m,\gamma_m}\sum_{i=1}^N L(y_i,\sum_{m=1}^M \beta_mb(x_i;\gamma_m))$$

- 前向分布算法(forward stagewise algorithm): 该算法的基本思路为: 由于学习是加法模型,如果可以从前往后,每一步只学习一个基函数及其系数,逐步逼近优化目标函数式(即损失函数极小化表达式),那么就可以简化优化的复杂度.具体地,每一步只需要优化如下损失函数:

$$\min_{\beta,\gamma}\sum_{i=1}^N L(y_i,\sum_{m=1}^M \beta_mb(x_i;\gamma_m))$$

### 4.2 基于前向分布算法的AdaBoost推导

　　前向分布算法学习的是加法模型,当基函数为基本分类器时,该加法模型等价于AdaBoost的最终分类器$f(x)=\sum_{m+1}^M \alpha_mG_m(x)$,它由基本分类器$G_m(x)$及其系数$\alpha_m$组成

　　前向分布算法逐一学习基函数,这一过程与AdaBoost算法逐一学习基本分类器的过程一致

　　下面证明前向分布算法的损失函数是指数函数$L(y,f(x))=\exp[-yf(x)]$时,其学习的具体操作等价于AdaBoost算法学习的具体操作

　　假设经过m-1轮迭代前向分布算法已经得到$f_{m-1}(x)$:

$$f_{m-1}(x)=f_{m-2}(x) + \alpha_{m-1}G_{m-1}(x)$$

$$=\alpha_1G_1 + \alpha_2G_2 + \cdots + \alpha_{m-1}G_{m-1}(x)$$

　　在m轮迭代得到$\alpha_m$,$G_m(x)$和$f_m(x)$,表示为

$$f_m(x)=f_{m-1}(x)+\alpha_mG_{m}(x)$$

　　此时参数$\alpha_m$和$G_m$均未知.因此,我们的目标是要得到最小化损失函数,通过最小化损失函数来得到模型中所需要的参数.而在AdaBoost算法中,每一轮都需要更新样本的权值参数,故而在每一轮的迭代中需要加工损失函数极小化,然后根据得到每个样例的权重更新参数.这样在每轮的迭代过程中只需要将当前基函数在训练集上的损失函数最小,最终使得$f_m(x)$在训练样本上的指数损失最小

$$(\alpha_m,G_m(x))=arg \min_{\alpha,G}\sum_{i=1}^N \exp[-y_i(f_{m-1}(x_i) + \alpha G(x_i))]$$

　　我们先假定$G_1(x),G_2(x),\cdots,G_{m-1}(x)$和$\alpha_1,\alpha_2,\cdots,\alpha_{m-1}$已知,求解$G_m(x)$和$\alpha_m$

　　可以将上式表示为

$$(\alpha_m,G_m(x))=arg \min_{\alpha,G}\sum_{i=1}^N \hat w_{mi} \exp[-y_i\alpha G(x_i)]$$

　　其中

$$\hat w_{mi}=\exp [-y_i f_{m-1}(x_i)]$$

因为$\hat w_{mi}$既不依赖$\alpha$也不依赖于G,所以与最小化无关.但它依赖于$f_{m-1}(x)$,随着每一次迭代而发生改变.现证使上式达到最小的$\alpha_m$和$G_m$就是AdaBoost算法所得到的$\alpha_m$和$G_m(x)$.求解可分为两步

- 1) 首先,求$G_m(x)$.对任意的$\alpha >0$,使指数损失函数最小的$G(x)$由下式得到

$$G_m(x)=\arg \min_G \sum_{i=1}^N \hat w_{mi}I(y_i \neq G(x_i))$$

　　此分类器$G_m^*(x)$即为AdaBoost算法的基本分类器$G_m(x)$,因为它是使第m轮加权训练数据分类误差率最小的基本分类器

- 2) 然后,求$\alpha_m$

$$\sum_{i=1}^N \hat w_{mi} \exp[-y_i \alpha G(x_i)] = \sum_{y_i=G_m(x_i)} \hat w_{mi}e^{-a}+\sum_{y_i \neq G_m(x_i)} \hat w_{mi}e^{a}$$

$$=e^a \sum_{y_i \neq G_m(x_i)}\hat w_{mi} - e^a \sum_{yi \neq G_m(x_i)}\hat w_{mi} + e^a \sum_{yi \neq G_m(x_i)}\hat w_{mi} + e^{-a}\sum_{y_i = G_m(x_i)}\hat w_{mi}$$

$$=(e^a - e^{-a})\sum_{i=1}^N \hat w_{mi}I(y_i \neq G(x_i)) + e^{-a}\sum_{i=1}^N \hat w_{mi}$$

将已求得的$G_m(x)$代入上式,对$\alpha$求导并使导数为0,即得到使其损失函数最小的$\alpha$.设

$$g(\alpha)=(e^a - e^{-a})\sum_{i=1}^N \hat w_{mi}I(y_i \neq G(x_i)) + e^{-a}\sum_{i=1}^N \hat w_{mi}$$

求导,并令其为0

$$\frac{\partial g(\alpha)}{\partial \alpha}=(e^a+e^{-a})\sum_{i=1}^N \hat w_{mi}I(y_i \neq G(x_i)) - e^{-\alpha}\sum_{i=1}^N \hat w_{mi}=0$$

得到

$$(e^a+e^{-a})\sum_{i=1}^N \hat w_{mi}I(y_i \neq G(x_i)) = e^{-\alpha}\sum_{i=1}^N \hat w_{mi}$$

$$(e^{2 \alpha}+1)\sum_{i=1}^N \hat w_{mi}I(y_i \neq G(x_i))=\sum_{i=1}^N \hat w_{mi}$$

$$\frac{(e^{2 \alpha}+1)\sum_{i=1}^N \hat w_{mi}I(y_i \neq G(x_i))}{\sum_{i=1}^N \hat w_{mi}}=1$$

令

$$e_m = \frac{\sum_{i=1}^N \hat w_{mi}I(y_i \neq G(x_i))}{\sum_{i=1}^N \hat w_{mi}}$$

$e_m$是分类误差率,得到

$$e_m(e^{2\alpha + 1})=1$$

最终得到使损失函数最小的$\alpha$

$$\alpha_m^*=\frac{1}{2}\log \frac{1-e_m}{e_m}$$

这就是之前我们的权重系数$\alpha_m$的来源

最后来看一下每一轮样本权值的更新

$$f_m(x)=f_{m-1}(x)+\alpha_mG_m(x)$$

以及

$$\hat w_{mi}=exp[-y_i f_{m-1}(x)]$$

可得

$$\hat w_{m+1,i}=\exp[-y_if_m(x)]=\exp[-y_i(f_{m-1}(x)+\alpha_mG_m(x))]$$

$$=\hat w_{mi}exp[-y_i\alpha_mG_m(x)]$$

从这一步中我们可以看到,这与开篇中所提到的AdaBoost的算法流程中的权重系数$(w_{m+1,i})$仅相差一个规范化因子$Z_m$,因而是等价的.